# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 12:40PM,241666,19,TR-RN-07132022,"GCH Granules USA, Inc.",Released
1,Jul 12 2022 12:28PM,241662,10,Enova-10281,Emerginnova,Released
2,Jul 12 2022 12:17PM,241605,15,8259985,"Brookfield Pharmaceuticals, LLC",Released
3,Jul 12 2022 12:17PM,241605,15,8260029,"Brookfield Pharmaceuticals, LLC",Released
4,Jul 12 2022 12:17PM,241605,15,8260030,"Brookfield Pharmaceuticals, LLC",Released
5,Jul 12 2022 12:17PM,241605,15,8260031,"Brookfield Pharmaceuticals, LLC",Executing
6,Jul 12 2022 12:17PM,241605,15,8260032,"Brookfield Pharmaceuticals, LLC",Released
7,Jul 12 2022 12:17PM,241605,15,8260033,"Brookfield Pharmaceuticals, LLC",Executing
8,Jul 12 2022 12:17PM,241605,15,8260034,"Brookfield Pharmaceuticals, LLC",Released
9,Jul 12 2022 12:17PM,241605,15,8260042,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,241654,Released,1
42,241656,Released,9
43,241657,Released,1
44,241662,Released,1
45,241666,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241654,NaN,NaN,1.0
241656,NaN,NaN,9.0
241657,NaN,NaN,1.0
241662,NaN,NaN,1.0
241666,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241562,0.0,7.0,13.0
241575,0.0,1.0,0.0
241593,3.0,0.0,1.0
241596,2.0,17.0,1.0
241597,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241562,0,7,13
241575,0,1,0
241593,3,0,1
241596,2,17,1
241597,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241562,0,7,13
1,241575,0,1,0
2,241593,3,0,1
3,241596,2,17,1
4,241597,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241562,,7,13
1,241575,,1,
2,241593,3,,1
3,241596,2,17,1
4,241597,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 12:40PM,241666,19,"GCH Granules USA, Inc."
1,Jul 12 2022 12:28PM,241662,10,Emerginnova
2,Jul 12 2022 12:17PM,241605,15,"Brookfield Pharmaceuticals, LLC"
59,Jul 12 2022 12:04PM,241657,10,"Senseonics, Incorporated"
60,Jul 12 2022 11:55AM,241656,10,"Methapharm, Inc."
69,Jul 12 2022 11:52AM,241654,10,"Citieffe, Inc."
70,Jul 12 2022 11:44AM,241649,20,Alumier Labs Inc.
71,Jul 12 2022 11:34AM,241652,19,ACG North America LLC
72,Jul 12 2022 11:34AM,241651,10,"Citieffe, Inc."
73,Jul 12 2022 11:34AM,241650,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 12 2022 12:40PM,241666,19,"GCH Granules USA, Inc.",,,1
1,Jul 12 2022 12:28PM,241662,10,Emerginnova,,,1
2,Jul 12 2022 12:17PM,241605,15,"Brookfield Pharmaceuticals, LLC",,6,51
3,Jul 12 2022 12:04PM,241657,10,"Senseonics, Incorporated",,,1
4,Jul 12 2022 11:55AM,241656,10,"Methapharm, Inc.",,,9
5,Jul 12 2022 11:52AM,241654,10,"Citieffe, Inc.",,,1
6,Jul 12 2022 11:44AM,241649,20,Alumier Labs Inc.,,,1
7,Jul 12 2022 11:34AM,241652,19,ACG North America LLC,,,1
8,Jul 12 2022 11:34AM,241651,10,"Citieffe, Inc.",,,1
9,Jul 12 2022 11:34AM,241650,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 12:40PM,241666,19,"GCH Granules USA, Inc.",1,,
1,Jul 12 2022 12:28PM,241662,10,Emerginnova,1,,
2,Jul 12 2022 12:17PM,241605,15,"Brookfield Pharmaceuticals, LLC",51,6,
3,Jul 12 2022 12:04PM,241657,10,"Senseonics, Incorporated",1,,
4,Jul 12 2022 11:55AM,241656,10,"Methapharm, Inc.",9,,
5,Jul 12 2022 11:52AM,241654,10,"Citieffe, Inc.",1,,
6,Jul 12 2022 11:44AM,241649,20,Alumier Labs Inc.,1,,
7,Jul 12 2022 11:34AM,241652,19,ACG North America LLC,1,,
8,Jul 12 2022 11:34AM,241651,10,"Citieffe, Inc.",1,,
9,Jul 12 2022 11:34AM,241650,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 12:40PM,241666,19,"GCH Granules USA, Inc.",1,,
1,Jul 12 2022 12:28PM,241662,10,Emerginnova,1,,
2,Jul 12 2022 12:17PM,241605,15,"Brookfield Pharmaceuticals, LLC",51,6,
3,Jul 12 2022 12:04PM,241657,10,"Senseonics, Incorporated",1,,
4,Jul 12 2022 11:55AM,241656,10,"Methapharm, Inc.",9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 12:40PM,241666,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Jul 12 2022 12:28PM,241662,10,Emerginnova,1.0,NaN,NaN
2,Jul 12 2022 12:17PM,241605,15,"Brookfield Pharmaceuticals, LLC",51.0,6.0,NaN
3,Jul 12 2022 12:04PM,241657,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Jul 12 2022 11:55AM,241656,10,"Methapharm, Inc.",9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 12:40PM,241666,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Jul 12 2022 12:28PM,241662,10,Emerginnova,1.0,0.0,0.0
2,Jul 12 2022 12:17PM,241605,15,"Brookfield Pharmaceuticals, LLC",51.0,6.0,0.0
3,Jul 12 2022 12:04PM,241657,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Jul 12 2022 11:55AM,241656,10,"Methapharm, Inc.",9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4832693,127.0,23.0,3.0
12,241645,1.0,0.0,0.0
15,241605,51.0,6.0,0.0
16,241601,1.0,0.0,0.0
19,1449756,5.0,1.0,3.0
20,966466,33.0,18.0,0.0
21,724899,3.0,0.0,0.0
22,241623,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4832693,127.0,23.0,3.0
1,12,241645,1.0,0.0,0.0
2,15,241605,51.0,6.0,0.0
3,16,241601,1.0,0.0,0.0
4,19,1449756,5.0,1.0,3.0
5,20,966466,33.0,18.0,0.0
6,21,724899,3.0,0.0,0.0
7,22,241623,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,127.0,23.0,3.0
1,12,1.0,0.0,0.0
2,15,51.0,6.0,0.0
3,16,1.0,0.0,0.0
4,19,5.0,1.0,3.0
5,20,33.0,18.0,0.0
6,21,3.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,127.0
1,12,Released,1.0
2,15,Released,51.0
3,16,Released,1.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
Executing,23.0,0.0,6.0,0.0,1.0,18.0,0.0,1.0
Released,127.0,1.0,51.0,1.0,5.0,33.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,23.0,0.0,6.0,0.0,1.0,18.0,0.0,1.0
2,Released,127.0,1.0,51.0,1.0,5.0,33.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Executing,23.0,0.0,6.0,0.0,1.0,18.0,0.0,1.0
2,Released,127.0,1.0,51.0,1.0,5.0,33.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()